<a href="https://colab.research.google.com/github/IshwaVen/Diamonds-price-prediction/blob/main/InvoiceParser_Qwen2.5-0.5B-Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_n21hmqz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_n21hmqz
  Resolved https://github.com/huggingface/transformers to commit 98adf24883b007c2a7fb17bab1c01b1614673433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!python -m pip install qwen-vl-utils

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import json

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct"
)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "file:////content/sample_data/invoice_1.jpg",
                "resized_height": 1753,
                "resized_width": 1240,
            },
            {
                "type": "text",
                # "text": "retrieve invoice items table data. return response in JSON format"
                # "text": "retrieve description and gross worth data from invoice items table. return response in JSON format"
                "text": "retrieve invoice_number, date_of_issue, seller_info, client_info, seller_tax_id, client_tax_id, iban, invoice_items_table, invoice_summary_table. return response in JSON format"
            }
        ]
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [ ]:
json_string = output_text[0]

json_string = json_string.strip("[]'")
json_string = json_string.replace("```json\n", "").replace("\n```", "")
json_string = json_string.replace("'", "")

try:
    formatted_json = json.loads(json_string)
    print(json.dumps(formatted_json, indent=2))
except json.JSONDecodeError as e:
    print("Failed to parse JSON:", e)

{
  "invoice_number": "61356291",
  "date_of_issue": "09/06/2012",
  "seller_info": {
    "name": "Chapman, Kim and Green",
    "address": "64731 James Branch, Smithmouth, NC 26872",
    "tax_id": "949-84-9105",
    "iban": "GB50ACIE59715038217063"
  },
  "client_info": {
    "name": "Rodriguez-Stevens",
    "address": "2280 Angela Plain, Hortonshire, MS 93248",
    "tax_id": "939-98-8477"
  },
  "invoice_items_table": [
    {
      "no": 1,
      "description": "Wine Glasses Goblets Pair Clear Glass",
      "quantity": 5,
      "unit": "each",
      "net_price": 12.0,
      "net_worth": 60.0,
      "vat": 10,
      "gross_worth": 66.0
    },
    {
      "no": 2,
      "description": "With Hooks Stemware Storage Multiple Uses Iron Wine Rack Hanging Glass",
      "quantity": 4,
      "unit": "each",
      "net_price": 28.08,
      "net_worth": 112.32,
      "vat": 10,
      "gross_worth": 123.55
    },
    {
      "no": 3,
      "description": "Replacement Corkscrew Parts Spiral Worm Wi